In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk 
import string
import re
import ast
# %matplotlib inline
# pd.set_option('display.max_colwidth', 100)

In [10]:
# Load dataset
def load_data():
    data = pd.read_csv('jokowi.csv')#ubah nama file sesai dengan nama file kalian
    return data

tweet_df = load_data()
tweet_df.head()

#definisi dataframe
df  = pd.DataFrame(tweet_df[['datetime','username', 'content']])#ubah dataframe sesuai column kalian



In [11]:
#====================================Cleansing=====================================#
#remove usero
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt    
df['remove_user'] = np.vectorize(remove_pattern)(df['content'], "@[\w]*")
df

,datetime,username,content,remove_user
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...,Parah emang...\nMalah bilang masyarakat haru...
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....,O... pantes dulu ada yang sampe nangis\nSe...
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...,Bangsa Indonesia dengan bapak Jokowi tetep di...
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...,Berkolaborasi dengan tiga kementrian tingkatka...
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...,Pemerintah tidak serius lakukan pemberantasa...
...,...,...,...,...
995,2022-09-07 20:24:03+00:00,TbDjahri,"@cobeh2021 Kabinet rusak byk mentri py kasus, ...","Kabinet rusak byk mentri py kasus, jokowi ema..."
996,2022-09-07 20:24:01+00:00,Christi28566166,@SisiwittFrida Menurutku tingginya itulah menu...,Menurutku tingginya itulah menunjukkan sistem...
997,2022-09-07 20:24:01+00:00,nganuNesia,@kanopimedia @YLBHI @amnestyindo @jokowi Polis...,"Polisi, kek anak lu gak ada yang jadi maha..."
998,2022-09-07 20:23:34+00:00,FrankhaStarla,@jokowi Itu mati Jokowi sama penjajah KITA BUK...,Itu mati Jokowi sama penjajah KITA BUKTIKAN I...


In [12]:
#  #=================================Stopword========================================#
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

def stop(text):
    factory = StopWordRemoverFactory()
    more_stopword = ['dengan', 'ia','bahwa','oleh']
    stopword= factory.create_stop_word_remover()
    out = stopword.remove(text)
    return out

df['stopword'] = df['remove_user'].apply(stop)
df

,datetime,username,content,remove_user,stopword
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...,Parah emang...\nMalah bilang masyarakat haru...,Parah emang...\nMalah bilang masyarakat jaga...
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....,O... pantes dulu ada yang sampe nangis\nSe...,O... pantes sampe nangis\nSekarang nyanyi-...
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...,Bangsa Indonesia dengan bapak Jokowi tetep di...,Bangsa Indonesia Jokowi tetep dipuja - puja b...
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...,Berkolaborasi dengan tiga kementrian tingkatka...,"Berkolaborasi kementrian tingkatkan wirausaha,..."
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...,Pemerintah tidak serius lakukan pemberantasa...,Pemerintah serius lakukan pemberantasan koru...
...,...,...,...,...,...
995,2022-09-07 20:24:03+00:00,TbDjahri,"@cobeh2021 Kabinet rusak byk mentri py kasus, ...","Kabinet rusak byk mentri py kasus, jokowi ema...","Kabinet rusak byk mentri py kasus, jokowi ema..."
996,2022-09-07 20:24:01+00:00,Christi28566166,@SisiwittFrida Menurutku tingginya itulah menu...,Menurutku tingginya itulah menunjukkan sistem...,Menurutku tingginya sistem demokrasi negeri h...
997,2022-09-07 20:24:01+00:00,nganuNesia,@kanopimedia @YLBHI @amnestyindo @jokowi Polis...,"Polisi, kek anak lu gak ada yang jadi maha...","Polisi, kek anak lu gak mahasiswa aja,"
998,2022-09-07 20:23:34+00:00,FrankhaStarla,@jokowi Itu mati Jokowi sama penjajah KITA BUK...,Itu mati Jokowi sama penjajah KITA BUKTIKAN I...,Itu mati Jokowi penjajah KITA BUKTIKAN ITU HU...


In [13]:
#=================================Tokenizing========================================#
#tokenize
from nltk.tokenize import TweetTokenizer

def token(tweet):
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    return tweet_tokens

df['token'] = df['stopword'].apply(token)
df

,datetime,username,content,remove_user,stopword,token
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...,Parah emang...\nMalah bilang masyarakat haru...,Parah emang...\nMalah bilang masyarakat jaga...,"[parah, emang, ..., malah, bilang, masyarakat,..."
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....,O... pantes dulu ada yang sampe nangis\nSe...,O... pantes sampe nangis\nSekarang nyanyi-...,"[o, ..., pantes, sampe, nangis, sekarang, nyan..."
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...,Bangsa Indonesia dengan bapak Jokowi tetep di...,Bangsa Indonesia Jokowi tetep dipuja - puja b...,"[bangsa, indonesia, jokowi, tetep, dipuja, -, ..."
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...,Berkolaborasi dengan tiga kementrian tingkatka...,"Berkolaborasi kementrian tingkatkan wirausaha,...","[berkolaborasi, kementrian, tingkatkan, wiraus..."
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...,Pemerintah tidak serius lakukan pemberantasa...,Pemerintah serius lakukan pemberantasan koru...,"[pemerintah, serius, lakukan, pemberantasan, k..."
...,...,...,...,...,...,...
995,2022-09-07 20:24:03+00:00,TbDjahri,"@cobeh2021 Kabinet rusak byk mentri py kasus, ...","Kabinet rusak byk mentri py kasus, jokowi ema...","Kabinet rusak byk mentri py kasus, jokowi ema...","[kabinet, rusak, byk, mentri, py, kasus, ,, jo..."
996,2022-09-07 20:24:01+00:00,Christi28566166,@SisiwittFrida Menurutku tingginya itulah menu...,Menurutku tingginya itulah menunjukkan sistem...,Menurutku tingginya sistem demokrasi negeri h...,"[menurutku, tingginya, sistem, demokrasi, nege..."
997,2022-09-07 20:24:01+00:00,nganuNesia,@kanopimedia @YLBHI @amnestyindo @jokowi Polis...,"Polisi, kek anak lu gak ada yang jadi maha...","Polisi, kek anak lu gak mahasiswa aja,","[polisi, ,, kek, anak, lu, gak, mahasiswa, aja..."
998,2022-09-07 20:23:34+00:00,FrankhaStarla,@jokowi Itu mati Jokowi sama penjajah KITA BUK...,Itu mati Jokowi sama penjajah KITA BUKTIKAN I...,Itu mati Jokowi penjajah KITA BUKTIKAN ITU HU...,"[itu, mati, jokowi, penjajah, kita, buktikan, ..."


In [14]:
def symclean(text):
    tokens = text
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha()]
    return words
df['remove_sym'] = df['token'].apply(symclean)
df

,datetime,username,content,remove_user,stopword,token,remove_sym
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...,Parah emang...\nMalah bilang masyarakat haru...,Parah emang...\nMalah bilang masyarakat jaga...,"[parah, emang, ..., malah, bilang, masyarakat,...","[parah, emang, malah, bilang, masyarakat, jaga..."
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....,O... pantes dulu ada yang sampe nangis\nSe...,O... pantes sampe nangis\nSekarang nyanyi-...,"[o, ..., pantes, sampe, nangis, sekarang, nyan...","[o, pantes, sampe, nangis, sekarang]"
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...,Bangsa Indonesia dengan bapak Jokowi tetep di...,Bangsa Indonesia Jokowi tetep dipuja - puja b...,"[bangsa, indonesia, jokowi, tetep, dipuja, -, ...","[bangsa, indonesia, jokowi, tetep, dipuja, puj..."
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...,Berkolaborasi dengan tiga kementrian tingkatka...,"Berkolaborasi kementrian tingkatkan wirausaha,...","[berkolaborasi, kementrian, tingkatkan, wiraus...","[berkolaborasi, kementrian, tingkatkan, wiraus..."
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...,Pemerintah tidak serius lakukan pemberantasa...,Pemerintah serius lakukan pemberantasan koru...,"[pemerintah, serius, lakukan, pemberantasan, k...","[pemerintah, serius, lakukan, pemberantasan, k..."
...,...,...,...,...,...,...,...
995,2022-09-07 20:24:03+00:00,TbDjahri,"@cobeh2021 Kabinet rusak byk mentri py kasus, ...","Kabinet rusak byk mentri py kasus, jokowi ema...","Kabinet rusak byk mentri py kasus, jokowi ema...","[kabinet, rusak, byk, mentri, py, kasus, ,, jo...","[kabinet, rusak, byk, mentri, py, kasus, jokow..."
996,2022-09-07 20:24:01+00:00,Christi28566166,@SisiwittFrida Menurutku tingginya itulah menu...,Menurutku tingginya itulah menunjukkan sistem...,Menurutku tingginya sistem demokrasi negeri h...,"[menurutku, tingginya, sistem, demokrasi, nege...","[menurutku, tingginya, sistem, demokrasi, nege..."
997,2022-09-07 20:24:01+00:00,nganuNesia,@kanopimedia @YLBHI @amnestyindo @jokowi Polis...,"Polisi, kek anak lu gak ada yang jadi maha...","Polisi, kek anak lu gak mahasiswa aja,","[polisi, ,, kek, anak, lu, gak, mahasiswa, aja...","[polisi, kek, anak, lu, gak, mahasiswa, aja]"
998,2022-09-07 20:23:34+00:00,FrankhaStarla,@jokowi Itu mati Jokowi sama penjajah KITA BUK...,Itu mati Jokowi sama penjajah KITA BUKTIKAN I...,Itu mati Jokowi penjajah KITA BUKTIKAN ITU HU...,"[itu, mati, jokowi, penjajah, kita, buktikan, ...","[itu, mati, jokowi, penjajah, kita, buktikan, ..."


In [15]:
#  #=================================Normalisasi========================================#
from modulenorm.modNormalize import normalize
from modulenorm.modTokenizing import tokenize

def spell(text_norm):
    usenorm = normalize()
    text_norm = usenorm.spellNormalize(text_norm) 
    return text_norm

df['normal'] = df['remove_sym'].apply(spell)
df

,datetime,username,content,remove_user,stopword,token,remove_sym,normal
0,2022-09-07 23:59:58+00:00,young_ladylike,@erikamelfa @jokowi Parah emang...\nMalah bila...,Parah emang...\nMalah bilang masyarakat haru...,Parah emang...\nMalah bilang masyarakat jaga...,"[parah, emang, ..., malah, bilang, masyarakat,...","[parah, emang, malah, bilang, masyarakat, jaga...","[parah, emang, malah, bilang, masyarakat, jaga..."
1,2022-09-07 23:59:54+00:00,SastroJoyo1,@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O.....,O... pantes dulu ada yang sampe nangis\nSe...,O... pantes sampe nangis\nSekarang nyanyi-...,"[o, ..., pantes, sampe, nangis, sekarang, nyan...","[o, pantes, sampe, nangis, sekarang]","[d, pantes, sampe, nangis, sekarang]"
2,2022-09-07 23:59:47+00:00,oketa214,@jokowi Bangsa Indonesia dengan bapak Jokowi t...,Bangsa Indonesia dengan bapak Jokowi tetep di...,Bangsa Indonesia Jokowi tetep dipuja - puja b...,"[bangsa, indonesia, jokowi, tetep, dipuja, -, ...","[bangsa, indonesia, jokowi, tetep, dipuja, puj...","[bangsat, indonesia, jokowi, tetep, dipuja, pu..."
3,2022-09-07 23:59:42+00:00,pahlevi0884,Berkolaborasi dengan tiga kementrian tingkatka...,Berkolaborasi dengan tiga kementrian tingkatka...,"Berkolaborasi kementrian tingkatkan wirausaha,...","[berkolaborasi, kementrian, tingkatkan, wiraus...","[berkolaborasi, kementrian, tingkatkan, wiraus...","[berkolaborasi, kementrian, tinggalkan, wiraus..."
4,2022-09-07 23:59:32+00:00,_ikhwanyan,@tvOneNews Pemerintah @jokowi tidak serius lak...,Pemerintah tidak serius lakukan pemberantasa...,Pemerintah serius lakukan pemberantasan koru...,"[pemerintah, serius, lakukan, pemberantasan, k...","[pemerintah, serius, lakukan, pemberantasan, k...","[pemerintah, serius, lakukan, pemberantasan, k..."
...,...,...,...,...,...,...,...,...
995,2022-09-07 20:24:03+00:00,TbDjahri,"@cobeh2021 Kabinet rusak byk mentri py kasus, ...","Kabinet rusak byk mentri py kasus, jokowi ema...","Kabinet rusak byk mentri py kasus, jokowi ema...","[kabinet, rusak, byk, mentri, py, kasus, ,, jo...","[kabinet, rusak, byk, mentri, py, kasus, jokow...","[kabinet, rusak, byk, menari, py, kamu, jokowi..."
996,2022-09-07 20:24:01+00:00,Christi28566166,@SisiwittFrida Menurutku tingginya itulah menu...,Menurutku tingginya itulah menunjukkan sistem...,Menurutku tingginya sistem demokrasi negeri h...,"[menurutku, tingginya, sistem, demokrasi, nege...","[menurutku, tingginya, sistem, demokrasi, nege...","[menurutku, cinggirnya, sista, demokrasi, nege..."
997,2022-09-07 20:24:01+00:00,nganuNesia,@kanopimedia @YLBHI @amnestyindo @jokowi Polis...,"Polisi, kek anak lu gak ada yang jadi maha...","Polisi, kek anak lu gak mahasiswa aja,","[polisi, ,, kek, anak, lu, gak, mahasiswa, aja...","[polisi, kek, anak, lu, gak, mahasiswa, aja]","[dposisi, kek, anak, lu, gak, mahasiswa, aja]"
998,2022-09-07 20:23:34+00:00,FrankhaStarla,@jokowi Itu mati Jokowi sama penjajah KITA BUK...,Itu mati Jokowi sama penjajah KITA BUKTIKAN I...,Itu mati Jokowi penjajah KITA BUKTIKAN ITU HU...,"[itu, mati, jokowi, penjajah, kita, buktikan, ...","[itu, mati, jokowi, penjajah, kita, buktikan, ...","[itu, mati, jokowi, penjajah, kita, buktikan, ..."


In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize

max_features = 1000

def join(text):
    string =' '.join(map(str,text))
    return string

df['final'] = df['normal'].apply(join)

def generate_tfidf_mat(min_gram, max_gram):
    cvect = CountVectorizer(max_features=max_features, ngram_range=(min_gram, max_gram))
    counts = cvect.fit_transform(df["final"])

    normalized_counts = normalize(counts, norm='l1', axis=1)

    tfidf = TfidfVectorizer(max_features=max_features, ngram_range=(min_gram, max_gram), smooth_idf=False)
    tfs = tfidf.fit_transform(df["final"])

    tfidf_mat = normalized_counts.multiply(tfidf.idf_).toarray()
    
    TF = normalized_counts.toarray()
    IDF = tfidf.idf_
    TF_IDF = tfidf_mat
    return TF, IDF, TF_IDF, tfidf.get_feature_names()

# ngram_range (1, 1) to use unigram only
tf_mat_unigram, idf_mat_unigram, tfidf_mat_unigram, terms_unigram = generate_tfidf_mat(1,1)

# ngram_range (2, 2) to use bigram only
tf_mat_bigram, idf_mat_bigram, tfidf_mat_bigram, terms_bigram = generate_tfidf_mat(2,2)

# ngram_range (3, 3) to use trigram only
tf_mat_trigram, idf_mat_trigram, tfidf_mat_trigram, terms_trigram = generate_tfidf_mat(3,3)

# ---------- check sparse data -------------------
idx_sample = 53

print("Show TFIDF sample ke-" + str(idx_sample), "\n")
print(df["final"][idx_sample], "\n")

print("\t\t\t", "TF", "\t\t", "IDF", "\t\t", "TF-IDF", "\t", "Term\n")
for i, item in enumerate(zip(tf_mat_unigram[idx_sample], idf_mat_unigram, tfidf_mat_unigram[idx_sample], terms_unigram)):
    if(item[2] != 0.0):
        print ("array position " + str(i) + "\t", 
               "%.6f" % item[0], "\t", 
               "%.6f" % item[1], "\t", 
               "%.6f" % item[2], "\t", 
               item[3])

Show TFIDF sample ke-53 

pakde kehadiran sebaiknya membubarken lembaga ad wow kak mengantri personil yang terlihat konspirasi korupsi di menginjak lah enforcement republik ini sedang terus ke tidak terendah bakat sukur terbalas 

			 TF 		 IDF 		 TF-IDF 	 Term

array position 3	 0.066667 	 6.809143 	 0.453943 	 ad
array position 186	 0.066667 	 3.918771 	 0.261251 	 di
array position 296	 0.066667 	 3.796881 	 0.253125 	 ini
array position 325	 0.066667 	 4.963316 	 0.330888 	 kak
array position 351	 0.066667 	 6.115996 	 0.407733 	 ke
array position 355	 0.066667 	 6.298317 	 0.419888 	 kehadiran
array position 418	 0.066667 	 4.816713 	 0.321114 	 korupsi
array position 438	 0.066667 	 5.605170 	 0.373678 	 lah
array position 455	 0.066667 	 6.521461 	 0.434764 	 lembaga
array position 645	 0.066667 	 5.961845 	 0.397456 	 pakde
array position 780	 0.066667 	 6.809143 	 0.453943 	 sedang
array position 896	 0.066667 	 7.214608 	 0.480974 	 terbalas
array position 912	 0.066667 	 5.1

In [72]:
# TfidfVectorizer 
# CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
# import ast

def join(text):
    string =' '.join(map(str,text))
    return string

df['final'] = df['normal'].apply(join)

# def join_text_list(texts):
#     texts = ast.literal_eval(texts)
#     return ' '.join([text for text in texts])
# df["tweet_join"] = df["normal"].apply(join_text_list)


# instantiate the vectorizer object
countvectorizer = CountVectorizer()
tfidfvectorizer = TfidfVectorizer()
# convert th documents into a matrix
count_wm = countvectorizer.fit_transform(df['final'])
tfidf_wm = tfidfvectorizer.fit_transform(df['final'])
#retrieve the terms found in the corpora
# if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , it will give same output of get_feature_names() methods)
#count_tokens = tfidfvectorizer.get_feature_names() # no difference
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_countvect = pd.DataFrame(data = count_wm.toarray(),index = df['content'] ,columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index =df['content'] ,columns = tfidf_tokens)
df_countvect
df_tfidfvect

,aamiin,abal,abang,abdullah,abis,abunawas,acakadut,ad,ada,adalah,...,yo,yoo,yoooiii,yu,yuk,yuu,yuuk,zaman,zhalim,zoliman
content,,,,,,,,,,,,,,,,,,,,,
@erikamelfa @jokowi Parah emang...\nMalah bilang masyarakat harus jaga data pribadi. Mau berkata kasar tapi ga bakal nyampe ke die.\n\nIncompetent!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
@Rizmaya__ @jokowi @PDemokrat @SBYudhoyono O... pantes dulu ada yang sampe nangis\nSekarang nyanyi-nyanyi 🤭,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
@jokowi Bangsa Indonesia dengan bapak Jokowi tetep dipuja - puja bangsa.. 🥰,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Berkolaborasi dengan tiga kementrian tingkatkan jumlah wirausaha,\nErick Thohir siap penuhi target dari presiden Jokowi.\n\n#BangkitBersamaET \n@erickthohir https://t.co/DMLZLBbmJX",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"@tvOneNews Pemerintah @jokowi tidak serius lakukan pemberantasan korupsi. Dari hal ini, sya pribadi menilai pemerintah secara terang-terangan medukung aksi korupsi. Penjahat Extraordinary Crime aja diberikan remisi. Maka tunggulah akan bermunculan koruptor-koruptor baru.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"@cobeh2021 Kabinet rusak byk mentri py kasus, jokowi emang bebal",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
@SisiwittFrida Menurutku tingginya itulah menunjukkan sistem demokrasi di negeri ini hidup mbak coba aja masa lalu apakah seperti ini bisa tapi yang korupsi mbak lihat saja saat itu .menurutku wajar saja tapi meski gimanapun aku sudah merasa pak jokowi sudah tepat meletakkan ibu srimulyani 👍👍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"@kanopimedia @YLBHI @amnestyindo @jokowi Polisi, kek anak lu gak ada yang jadi mahasiswa aja,",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

# banyaknya term yang akan digunakan, 
# di pilih berdasarkan top max_features 
# yang diurutkan berdasarkan term frequency seluruh corpus
max_features = 1000

def join(text):
    string =' '.join(map(str,text))
    return string

df['final'] = df['normal'].apply(join)
# Feature Engineering 
print ("------- TF-IDF on Tweet data -------")

tf_idf = TfidfVectorizer(max_features=max_features, binary=True)
tfidf_mat = tf_idf.fit_transform(df['final']).toarray()
terms = tf_idf.get_feature_names()
sums = tfidf_mat.sum(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append((term, sums[col] ))
    
print("TF-IDF ", type(tfidf_mat), tfidf_mat.shape)


col = pd.DataFrame(data, columns=['term','tf_idf'])
col

------- TF-IDF on Tweet data -------
TF-IDF  <class 'numpy.ndarray'> (1000, 1000)


C:\Users\faizi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,tf_idf
0,aamiin,1.183573
1,abdullah,1.755397
2,abis,1.362919
3,ad,0.877548
4,ada,4.288659
...,...,...
995,yntkts,0.699584
996,yoo,0.795381
997,yu,0.805953
998,yuu,0.972059
